# Grover's Algorithm

This implementation does phase inversion without requiring an additional qubit

# Imports

In [ ]:
import qckt
from qckt.backend import Qeng, Qdeb
import random as rnd
import numpy as np

# Qubits assignment for the algorithm

In [ ]:
marker_nqubits = 4
inpreg = [i for i in reversed(range(marker_nqubits))]
nqubits = marker_nqubits

# The 'needle' in the haytack to be searched = marker

In [ ]:
marked = int(rnd.random()*2**nqubits)
print(("Marker to search = {0:0"+str(nqubits-1)+"b}, ({0:d})").format(marked))

# The oracle circuit

In [ ]:
### Uf - The oracle (the verifying function) 
uf_ckt = qckt.QCkt(nqubits,name="Uf Invert")
zeros = []
for i in range(len(inpreg)):
	if (marked & (1 << i)) == 0:
		zeros.append(inpreg[-i-1])
uf_ckt.Border()
if len(zeros) > 0:
	uf_ckt.X(zeros)
uf_ckt.CZ(*inpreg)
if len(zeros) > 0:
	uf_ckt.X(zeros)
uf_ckt.Border()
uf_ckt.draw()

# The amplification (diffuser) circuit

In [ ]:
### amplify
ampckt = qckt.QCkt(nqubits,name="Amplify")
ampckt.H(inpreg)
ampckt.X(inpreg)
ampckt.CZ(*inpreg)
ampckt.X(inpreg)
ampckt.H(inpreg)
ampckt.draw()

# The initializer circuit

In [ ]:
### Initialize
initckt = qckt.QCkt(nqubits,name="Initialize")
initckt.H(inpreg)
initckt.draw()

# Assemble the full Grover's algorithm circuit

In [ ]:
fullckt = qckt.QCkt(nqubits=nqubits,nclbits=nqubits,name="Full Circuit")
fullckt = fullckt.append(initckt)
numitrs = int((np.pi/4.0)*(2.0**(nqubits/2.0))) # optimal # iter, less or more dont work
print("INVERT(Uf)-AMPLIFY iterations = ",numitrs)
for i in range(numitrs):
	fullckt = fullckt.append(uf_ckt)
	# fullckt.Probe('Invert',probestates=[marked-1, marked, marked+1])
	fullckt = fullckt.append(ampckt)
	# fullckt.Probe('Iteration {:d}'.format(i+1),probestates=[marked-1, marked, marked+1])
# fullckt.M(inpreg)  # use only if not using multiple shots
fullckt.draw()

# Run the circuit multiple times, display the stats of readouts

In [ ]:
job = qckt.Job(fullckt, shots=100)
bk = Qeng()  # to run multiple shots, and see stats of readouts
# bk = Qdeb()  # to see Probe on state of interest
bk.runjob(job)
_ = job.plot_counts()